In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/IMDB Dataset.csv'

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(file_path)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
df = df.drop_duplicates()
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.1 MB/s eta 0:00:00


In [ ]:
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
def expand_contractions(text):
  return contractions.fix(text)
def preprocess_text(text):

  wl = WordNetLemmatizer()

  soup = BeautifulSoup(text, "html.parser")
  text = soup.get_text()
  text = expand_contractions(text)
  emoji_clean = re.compile(
      "["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      "]+", flags = re.UNICODE
  )
  text = emoji_clean.sub(r'', text)
  text = re.sub(r'\.(?=\S)', '. ', text)
  text = re.sub(r'http\S+', '', text)
  text = "".join([
      word.lower() for word in text if word not in string.punctuation
  ])
  text = " ".join([
      wl.lemmatize(word) for word in text.split() if word not in stop and word.isalpha()
  ])
  return text

df['review'] = df['review'].apply(preprocess_text)

<ipython-input-12-faf08c880dff>:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
<ipython-input-12-faf08c880dff>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(preprocess_text)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
import numpy as np

In [ ]:
max_words = 10000
max_len = 150
tokenizer = Tokenizer(num_words=max_words)

In [ ]:
tokenizer.fit_on_texts(df['review'])
sequences = tokenizer.texts_to_sequences(df['review'])
X = pad_sequences(sequences, maxlen=max_len)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['sentiment'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(39665, 150)

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype("int32")

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1

In [ ]:

API = 'e055257d77c953d4c2b2ccdac08009973a34b4fe'

In [ ]:
!pip install wandb

In [ ]:
import wandb
import random
'''from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint'''

'from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint'

In [ ]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
! pip install gensim

In [ ]:
from gensim.models import Word2Vec

In [ ]:
sentences = [review.split() for review in df['review']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
embedding_matrix = np.zeros((max_words, 100))
for word, i in tokenizer.word_index.items():
  if i < max_words:
    try:
      embedding_matrix[i] = word2vec_model.wv[word]
    except KeyError:
      embedding_matrix[i] = np.zeros(embedding_dim)

In [ ]:
NUM_FILTERS = 128
KERNEL_SIZE = 5
from tensorflow.keras.optimizers import Adam
EPOCHS = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.0001
OPTIMIZER = Adam(learning_rate=LEARNING_RATE)


In [ ]:
wandb.init(
    project="cnn-wordvec",
    config={
        "metric": "accuracy",
        "epoch": EPOCHS,
        "batch_size": BATCH_SIZE
        }
)

train_accuracy,▁▄▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train_loss,█▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_accuracy,▁▅▆▆▇▇▇▆▇▇▇▆▆▇███▇▇▇
val_loss,█▅▅▃▃▃▃▃▃▂▂▃▃▂▁▁▁▂▁▂
train_accuracy,0.92565
train_loss,0.18337
val_accuracy,0.86831
val_loss,0.31098


In [ ]:
model_word2vec_cnn = Sequential([
    layers.Embedding(input_dim=max_words, output_dim=100, input_length=max_len, weights=[embedding_matrix], trainable=False),
    layers.Conv1D(NUM_FILTERS, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.5),
    layers.Conv1D(NUM_FILTERS, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.5),
    layers.Conv1D(NUM_FILTERS, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model_word2vec_cnn.compile(loss='binary_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

In [ ]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")

    # Train the model for one epoch and get the training metrics
    history = model_word2vec_cnn.fit(
        X_train, y_train,
        epochs=1,  # Train for one epoch
        batch_size=wandb.config.batch_size,
        validation_data=(X_test, y_test),
        verbose=1
    )
    wandb.log({
        "train_loss": history.history['loss'][0],
        "train_accuracy": history.history['accuracy'][0],
        "val_loss": history.history['val_loss'][0],
        "val_accuracy": history.history['val_accuracy'][0]
    })

    # Optionally, print metrics after each epoch
    print(f"Train Loss: {history.history['loss'][0]:.4f}")
    print(f"Train Accuracy: {history.history['accuracy'][0]:.4f}")
    print(f"Validation Loss: {history.history['val_loss'][0]:.4f}l")
    print(f"Validation Accuracy: {history.history['val_accuracy'][0]:.4f}")

Epoch 1/20
310/310 ━━━━━━━━━━━━━━━━━━━━ 118s 365ms/step - accuracy: 0.5645 - loss: 0.7435 - val_accuracy: 0.7989 - val_loss: 0.4579
Train Loss: 0.6356
Train Accuracy: 0.6446
Validation Loss: 0.4579l
Validation Accuracy: 0.7989
Epoch 2/20
310/310 ━━━━━━━━━━━━━━━━━━━━ 112s 362ms/step - accuracy: 0.7876 - loss: 0.4621 - val_accuracy: 0.8267 - val_loss: 0.4039
Train Loss: 0.4428
Train Accuracy: 0.7980
Validation Loss: 0.4039l
Validation Accuracy: 0.8267
Epoch 3/20
310/310 ━━━━━━━━━━━━━━━━━━━━ 114s 367ms/step - accuracy: 0.8170 - loss: 0.4077 - val_accuracy: 0.8329 - val_loss: 0.3923
Train Loss: 0.3988
Train Accuracy: 0.8228
Validation Loss: 0.3923l
Validation Accuracy: 0.8329
Epoch 4/20
310/310 ━━━━━━━━━━━━━━━━━━━━ 113s 363ms/step - accuracy: 0.8314 - loss: 0.3850 - val_accuracy: 0.8388 - val_loss: 0.3835
Train Loss: 0.3793
Train Accuracy: 0.8339
Validation Loss: 0.3835l
Validation Accuracy: 0.8388
Epoch 5/20
310/310 ━━━━━━━━━━━━━━━━━━━━ 115s 370ms/step - accuracy: 0.8401 - loss: 0.3696 - 

In [ ]:
model_word2vec_cnn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 150, 100)            │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_9 (Conv1D)                    │ (None, 146, 128)            │          64,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_9 (MaxPooling1D)       │ (None, 73, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 73, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_10 (Conv1D)                   │ (None, 69, 128)             │          82,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_10 (MaxPooling1D)      │ (None, 34, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 34, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 30, 128)             │          82,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_11 (MaxPooling1D)      │ (None, 15, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 1920)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │           1,921 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,690,437 (6.45 MB)

 Trainable params: 230,145 (899.00 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

 Optimizer params: 460,292 (1.76 MB)

In [ ]:
print("\nPerformance for Word2Vec + CNN:")
evaluate_model(model_word2vec_cnn, X_test, y_test)


Performance for Word2Vec + CNN:
310/310 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step
Accuracy: 0.8683
Precision: 0.8908
Recall: 0.8407
F1 Score: 0.8650


(0.8683069476656247,
 0.8908045977011494,
 0.8406990759341101,
 0.8650268706076891)